In [120]:
import cv2
import numpy as np
import PIL
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime
import os
import glob
import imutils

In [122]:
def resize2SquareKeepingAspectRation(img, size, interpolation):
    h, w = img.shape[:2]
    c = None if len(img.shape) < 3 else img.shape[2]
    if h == w: return cv2.resize(img, (size[0], size[1]), interpolation)
    if h > w: dif = h
    else:     dif = w
    x_pos = int((dif - w)/2.)
    y_pos = int((dif - h)/2.)
    if c is None:
        mask = np.zeros((dif, dif), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]
    else:
        mask = np.zeros((dif, dif, c), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w, :] = img[:h, :w, :]
    print(mask)
    return cv2.resize(mask, (size[0], size[1]), interpolation)

In [ ]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    
#     elif width is not None and height is not None:
#         r = width / float(w)
#         hh= int(h*r)
#         r = height /float(h)
#         ww = int(w * r)
#         dim =(ww,hh)
        
    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [ ]:
folderPath = r"/home/transferlearningtoolkit/tlt-experiments/Data/Data/differentres/training"
images = os.path.join(folderPath, "image_2")
labels = os.path.join(folderPath, "label_2")
convertedimages = os.path.join(folderPath, "convertedimage_2")
convertedlabel = os.path.join(folderPath, "convertedlabel_2")
if not os.path.exists(convertedimages):
    os.makedirs(convertedimages)
if not os.path.exists(convertedlabel):
    os.makedirs(convertedlabel)

In [ ]:
imageFiles = os.listdir(images)
labelFiles = os.listdir(labels)

In [ ]:
print(images)
print(labels)

In [ ]:
def drawBox(boxes, image):
    print(boxes)
    for i in range(0, len(boxes)):
        cv2.rectangle(image, (boxes[i][2], boxes[i][3]), (boxes[i][4], boxes[i][5]), (255, 0, 0), 1)
    plt.imshow(image)
    plt.show()

In [ ]:
def resize_with_padding(img, expected_size):
    print(img.size)
    img.thumbnail((expected_size[0], expected_size[1]))
    delta_width = expected_size[0] - img.size[0]
    delta_height = expected_size[1] - img.size[1]
    print(delta_width)
    print(delta_height)
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    print(pad_width)
    print(pad_height)
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding)

In [ ]:
targetSize1 = 480
targetSize2 = 272
size = (targetSize1,targetSize2)
for image in imageFiles:
    if image[:-4]+".txt" not in labelFiles:
        print(labels)
        print(image[:-4])
        continue
    imageName = os.path.join(images,image)
    labelFile = os.path.join(labels,image[:-4]+".txt")
    convertedimageFile = os.path.join(convertedimages,image)
    convertedlabelFile = os.path.join(convertedlabel,image[:-4]+".txt")
    imageToPredict = cv2.imread(imageName)
    #print(imageToPredict.shape)
    x_ = imageToPredict.shape[1]
    y_ = imageToPredict.shape[0]
    image = resize2SquareKeepingAspectRation(imageToPredict, size, cv2.INTER_AREA)
#     image = image_resize(imageToPredict, width = 480,height=272)
    #print(labelFile)
    targetSize1= image.shape[1]
    targetSize2= image.shape[0]
    print(image.shape)
#     riiesized = resize2SquareKeepingAspectRation(imageToPredict, size, cv2.INTER_AREA)
#     img = cv2.resize(imageToPredict, (targetSize1, targetSize2));
    x_scale = targetSize1 / x_
    y_scale = targetSize2 / y_
    #print(img.shape)
    img = np.array(image);
    file = open(convertedlabelFile, 'w')
    cv2.imwrite(convertedimageFile,img)
    with open(labelFile, 'r') as reader:
        line = reader.readline()
        bboxes=[]
        while line != '':  # The EOF char is an empty string
            # print(line, end='\n')
            lists=line.split( )
            name = lists[0]
            bbox =lists[4:8]
            bbox_ = [int(float(i)) for i in bbox]
            # original frame as named values
            xmin = int(np.round(bbox_[0] * x_scale))
            ymin = int(np.round(bbox_[1] * y_scale))
            xmax = int(np.round(bbox_[2] * x_scale))
            ymax = int(np.round(bbox_[3] * y_scale))
            # Box.drawBox([[1, 0, x, y, xmax, ymax]], img)
            drawBox([[1, 0, xmin, ymin, xmax, ymax]], img)
            bboxes.append(bbox_)
            writte =  f"{name} 0.00 0 0.0 {xmin}.00 {ymin}.00 {xmax}.00 {ymax}.00 0.0 0.0 0.0 0.0 0.0 0.0 0.0"
            file.write(writte)
            line = reader.readline()
            if line !='':
                file.write("\n")
    file.close()

In [ ]:
# im_path = r"/home/facit/Desktop/peter-broomfield-m3m-lnR90uM-unsplash.jpg"
# im_pth = r"/home/facit/Downloads/Padded-Image-Resizer-master/Input/image (38).jpg"

# img = cv2.imread(im_pth)
# size= (480,272)
# image = image_resize(img, width = 480,height=272)
# plt.imshow(image)